In [188]:
import pandas as pd
import numpy as np
import cdsapi
import utm
import os 
import json
import sys

In [189]:
df = pd.read_excel("สำรวจโรคยางพาราน่าน 67.xlsx")

In [190]:
df.head()

,ลำดับ,ชื่อเกษตรกร,วันที่สำรวจ,ตำบล,อำเภอ,จังหวัด,รหัสแปลง(code),พิกัด,Unnamed: 8,อายุต้นยาง(ปี),พันธุ์ยางพารา,โรคที่พบ,เปอร์เซ็นต์การเกิดโรค,ระดับความรุนแรง
0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaT,NaN,NaN,NaN,NaN,x,y,NaN,NaN,NaN,NaN,NaN
2,1.0,นายสมภาค หล้ากัน,2024-03-21,ปิงหลวง,นาหมื่น,น่าน,67NAN06_1,-,-,3,RRIM600,โรคใบร่วงชนิดใหม่,0.1,1.0
3,2.0,นายดุสิษฐ เสนนะ,2024-03-21,ปิงหลวง,นาหมื่น,น่าน,67NAN06_2,662742,2011966,3,RRIT251,โรคใบร่วงชนิดใหม่,0.6,3.0
4,3.0,นางวรรณพร โพธิยะ,2024-03-21,ปิงหลวง,นาหมื่น,น่าน,67NAN06_3,662661,2011893,4,RRIT251,โรคใบร่วงชนิดใหม่,0.4,3.0


In [191]:
df = df.iloc[2:, [1, 2, 3, 4, 5, 7, 8]]
df = df.rename({'วันที่สำรวจ': 'date', 'พิกัด': 'x', 'Unnamed: 8': 'y'}, axis=1)
df = df[df['x'] != '-']
df = df.reset_index(drop=True)


In [192]:
def adjust_year_to_gregorian(date_str):
    date_obj = pd.to_datetime(date_str, format='%d/%m/%y')
    gregorian_year = date_obj.year
    # Recreate the date with the adjusted year
    return date_obj.replace(year=gregorian_year).strftime('%Y-%m-%d')

df['date'] = df['date'].apply(adjust_year_to_gregorian)

In [193]:
df

,ชื่อเกษตรกร,date,ตำบล,อำเภอ,จังหวัด,x,y
0,นายดุสิษฐ เสนนะ,2024-03-21,ปิงหลวง,นาหมื่น,น่าน,662742,2011966
1,นางวรรณพร โพธิยะ,2024-03-21,ปิงหลวง,นาหมื่น,น่าน,662661,2011893
2,นายเหรียญ สมนา,2024-03-22,ยอด,สองแคว,น่าน,669332,2148196
3,นายจารุกิตติ์ ไชยสลี,2024-03-22,ยอด,สองแคว,น่าน,668766,2149595
4,นางสาวศุภลักษณ์ สุขสาด,2024-03-22,เตาปูน,สองแคว,แพร่,669168,2147887
5,นางบุญธรรม ชาวริม,2024-03-22,ยอด,สองแคว,น่าน,668365,2148569
6,นางบัวใย ลำคำ,2024-03-22,ยอด,สองแคว,น่าน,668709,2148225


In [194]:
def utmToLatLon(x:list, y:list, zone:int = 47) -> tuple:
    x, y = np.mean(x), np.mean(y) #find mid point
    return utm.to_latlon(x,y, zone_number=zone, zone_letter='N') #return (lat, lon)

In [195]:
DIR_OUTPUT = f'../Nan/1nc/'
os.makedirs(DIR_OUTPUT, exist_ok=True)
c = cdsapi.Client()
for i , data in df.iterrows():
    lat, lon = utmToLatLon(x=[data['x']], y=[data['y']], zone=47)
    year = int(data['date'].split('-')[0])
    month = int(data['date'].split('-')[1])
    day = int(data['date'].split('-')[2])
    print(f"\nDownloading {data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']} ({lat}, {lon})")
    c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                    '2m_temperature', 'downward_uv_radiation_at_the_surface', 'evaporation',
                    'soil_temperature_level_1', 'surface_pressure', 'total_precipitation',
                ],
                'year': year,
                'month': [str(i) for i in range(month-1, month+2)],
                'day': [str(i) for i in range(1, 32)],
                'area' : [lat, lon, lat, lon],                  
                'time': [
                    '00:00', '06:00', '12:00',
                    '18:00',
                ],
            },
            f"{DIR_OUTPUT}{i}-{data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']}.nc")

2024-08-08 15:35:09,340 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-08 15:35:09,341 WARNING MOVE TO CDS-Beta
2024-08-08 15:35:09,342 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-08-08 15:35:09,730 INFO Request is queued
2024-08-08 15:35:19,116 INFO Request is running
2024-08-08 15:39:32,346 INFO Request is completed
2024-08-08 15:39:32,346 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data2/adaptor.mars.internal-1723106356.5055993-25802-1-6157c759-ec9c-4e00-9a07

2024-08-08 15:39:35,776 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-08 15:39:35,776 WARNING MOVE TO CDS-Beta
2024-08-08 15:39:35,776 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-08-08 15:39:36,140 INFO Request is queued
2024-08-08 15:42:30,607 INFO Request is running
2024-08-08 15:48:01,980 INFO Request is completed
2024-08-08 15:48:01,981 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data8/adaptor.mars.internal-1723106762.5514598-18388-9-afc4608a-39ae-4d3a-bf6a

2024-08-08 15:48:05,245 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-08 15:48:05,247 WARNING MOVE TO CDS-Beta
2024-08-08 15:48:05,249 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-08-08 15:48:05,639 INFO Request is queued
2024-08-08 15:48:06,957 INFO Request is running
2024-08-08 15:54:29,461 INFO Request is completed
2024-08-08 15:54:29,462 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data9/adaptor.mars.internal-1723107146.7879066-27538-8-b343e11c-8f5e-4f9a-a15b

2024-08-08 15:54:32,019 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-08 15:54:32,021 WARNING MOVE TO CDS-Beta
2024-08-08 15:54:32,022 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-08-08 15:54:32,309 INFO Request is queued
2024-08-08 15:54:37,801 INFO Request is running
2024-08-08 16:00:55,917 INFO Request is completed
2024-08-08 16:00:55,917 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data9/adaptor.mars.internal-1723107601.9089067-11445-1-b7bdeb0d-3713-416

2024-08-08 16:00:59,476 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-08 16:00:59,477 WARNING MOVE TO CDS-Beta
2024-08-08 16:00:59,478 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-08-08 16:00:59,800 INFO Request is queued
2024-08-08 16:02:56,472 INFO Request is running
2024-08-08 16:07:24,453 INFO Request is completed
2024-08-08 16:07:24,454 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data9/adaptor.mars.internal-1723107980.4668975-9829-4-f24fbf6b-6164-4ccf-a1af-

2024-08-08 16:07:27,891 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-08 16:07:27,892 WARNING MOVE TO CDS-Beta
2024-08-08 16:07:27,893 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-08-08 16:07:28,152 INFO Request is queued
2024-08-08 16:07:31,142 INFO Request is running
2024-08-08 16:13:52,244 INFO Request is completed
2024-08-08 16:13:52,246 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data1/adaptor.mars.internal-1723108366.3523102-27099-14-d1660896-36c7-41

2024-08-08 16:13:55,648 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-08 16:13:55,649 WARNING MOVE TO CDS-Beta
2024-08-08 16:13:55,649 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-08-08 16:13:55,902 INFO Request is queued
2024-08-08 16:13:58,874 INFO Request is running
2024-08-08 16:18:17,727 INFO Request is completed
2024-08-08 16:18:17,729 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data5/adaptor.mars.internal-1723108671.098807-16466-13-7db3bab7-09b5-4f4